In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

import warnings #To make things cleaner...
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
# from xgboost import plot_importance

import pickle

In [3]:
df = pd.read_csv('../data/cleaned_train.csv')
df.drop('person_id', axis=1, inplace=True)
print(df.shape)
df.head()

(100, 17)


,gender,age,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),heart_rate_(bpm),daily_steps,occupation_office_worker,occupation_retired,occupation_student,bmi_category_obese,bmi_category_overweight,bmi_category_underweight,blood_pessure_sys,blood_pessure_dia,sleep_disorder
0,1,62,5.803,9.684,26.362,9.247,88.435,12135.856,0,0,1,0,1,0,131,78,No Disorder
1,0,35,7.928,4.590,111.303,3.092,62.090,19871.475,0,0,0,0,1,0,117,69,Insomnia
2,0,50,4.426,6.810,28.560,5.168,94.423,18754.661,1,0,0,1,0,0,135,86,No Disorder
3,1,38,9.032,4.683,76.912,6.037,80.149,12658.284,1,0,0,0,1,0,119,69,No Disorder
4,0,34,8.719,6.730,11.742,2.030,74.927,17742.211,0,0,1,0,0,0,117,69,Insomnia


## Round I - Combining Insomnia with Sleep Apnea

Really wouldn't want that to happen to me (if that's even possible). Regardless, let's start with that.

In [4]:
# unique_categories = df['target'].unique()
# print(unique_categories)
# category_to_int = {category: idx for idx, category in enumerate(unique_categories)}
# df['target_encoded'] = df['target'].map(category_to_int)

df['sleep_disorder_binary'] = [0 if value == 'No Disorder' else 1 for value in df['sleep_disorder']]
# df['sleep_disorder'].value_counts(normalize=True)

In [6]:
X = df.drop(['sleep_disorder', 'sleep_disorder_binary'], axis = 1) #Starting out, at least, with 'everything'.
y = df['sleep_disorder_binary']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.15, #Perhaps subject to change, but we'll go with it for now
                                                    random_state = 26, #Recall that I like this number
                                                    stratify=y) #Lest a dispropportionate amount of one be in any cut.

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [9]:
#Spoiler alert: I commented out most of the code as it did not do well at all (see conclusion below). However, I'll at least keep the best:

#Generally I'll only preserve the last bit of parameters I tried in this section of the code, not bothering to record every single iteration...

# pipe = Pipeline([
#     ('lr', LogisticRegression(max_iter=2500))
# ])

# pipe_params = {'lr__C' : [.85, .84, .86]
#               }

# gs_lrcv = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=5)

# gs_lrcv.fit(X_train, y_train)

# print(gs_lrcv.score(X_train, y_train), gs_lrcv.score(X_test, y_test))
# print(gs_lrcv.best_score_)
# print(gs_lrcv.best_params_)

#Hmm, not looking so hot. Then again, this is only the first model with our gross assumptions (re. combining them).

0.7529411764705882 0.6
0.6705882352941176
{'lr__C': 0.85}


In [11]:
# pipe = Pipeline([
#     ('rfc', RandomForestClassifier(random_state=26))
# ])

# pipe_params = {'rfc__min_samples_split' : [2,3,5]
#                ,'rfc__min_samples_leaf' : [1,2,3]
#                ,'rfc__max_features' : ['sqrt', 'log2']
#               }
# #Will keep grid search to make things easier
# gs_rfc = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=None)

# gs_rfc.fit(X_train, y_train)

# print(gs_rfc.score(X_train, y_train), gs_rfc.score(X_test, y_test))
# print(gs_rfc.best_score_)
# print(gs_rfc.best_params_)

1.0 0.6666666666666666
0.6823529411764706
{'rfc__max_features': 'sqrt', 'rfc__min_samples_leaf': 1, 'rfc__min_samples_split': 5}


In [12]:
# pipe = Pipeline([
#     ('svc', SVC())
# ])

# pipe_params = {'svc__C' : [1.0, .95, .9]
#                ,'svc__kernel' : ['rbf']
#                ,'svc__degree' : [3] #Oh, recall this only matters if your kernel is poly.
#                #,'svc__' : []
#               }
# #Will keep grid search to make things easier
# gs_svc = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=3)

# gs_svc.fit(X_train, y_train)

# print(gs_svc.score(X_train, y_train), gs_svc.score(X_test, y_test))
# print(gs_svc.best_score_)
# print(gs_svc.best_params_)

0.8470588235294118 0.6
0.69376026272578
{'svc__C': 1.0, 'svc__degree': 3, 'svc__kernel': 'rbf'}


In [13]:
#To at least make some actual progress today I'll see if KNN gives anything useful:

pipe = Pipeline([
    ('knn', KNeighborsClassifier())
])

pipe_params = {'knn__n_neighbors' : [17,13,9]
               ,'knn__p' : [1,2,3]
#                ,'knn__' : []
#                ,'knn__' : []
#                ,'knn__' : []
              }
#Will keep grid search to make things easier
gs_knn = GridSearchCV(pipe,
                  param_grid=pipe_params,
                  cv=3)

gs_knn.fit(X_train, y_train)

print(gs_knn.score(X_train, y_train), gs_knn.score(X_test, y_test))
print(gs_knn.best_score_)
print(gs_knn.best_params_)

0.7058823529411765 0.6
0.7175697865353037
{'knn__n_neighbors': 17, 'knn__p': 2}


In [14]:
# xgb = XGBClassifier(random_state=26)

# param_grid = {
#     'learning_rate': [.3, .4, .5]
#     ,'n_estimators': [150, 200, 250]
#     ,'max_depth': [2, 3, 4]
#     ,'subsample': [.9]
#     ,'colsample_bytree': [1.0]
#     ,'reg_alpha': [.004]
#     ,'reg_lambda': [1.25]
#     ,'min_child_weight': [1]
#     ,'gamma': [0]
# }

# gs_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, 
#                            scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# gs_xgb.fit(X_train, y_train)

# y_pred = gs_xgb.predict(X_test)

# print(gs_xgb.best_params_)
# print(gs_xgb.best_score_)
# print(accuracy_score(y_test, y_pred))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.3, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 200, 'reg_alpha': 0.004, 'reg_lambda': 1.25, 'subsample': 0.9}
0.6703612479474549
0.5333333333333333


## Round I - Conclusion

From the paltry amount that I did, quite pleasing that KNN is rising to the challenge! With a best scoe, highest of all of them of .72!

However, this is terrible. Recall that the baseline is 70%.... we might as tell everybody they're fine at this rate.

Perforce, it seems that our attempts to be lazy and lump everything together is not fruitful.

Now, perhaps our small sample size is to blame. Nevertheless, let's try to solve this as intended (multi-classification). Yet, bear in mind the need to deal with undersamplling somehow.

In [ ]:
stopstop

#Now readjust bros

# unique_categories = df['target'].unique()
# print(unique_categories)
# category_to_int = {category: idx for idx, category in enumerate(unique_categories)}
# df['target_encoded'] = df['target'].map(category_to_int)

df['sleep_disorder_binary'] = [0 if value == 'No Disorder' else 1 for value in df['sleep_disorder']]
# df['sleep_disorder'].value_counts(normalize=True)